In [1]:
#Get text file and save to BeautifulSoup Object
import requests

fin=open('nj_roster.txt','rt')
text=fin.read()
fin.close()

from bs4 import BeautifulSoup
soup=BeautifulSoup(text,"html5lib")



In [2]:
#Get list of dictionaries for each candidate
import re

def tag_BIO(child):
    return child.has_attr('href') and re.search('BIO\.asp\?Leg=\d+',child['href'])

root='https://www.njleg.state.nj.us/members/'

In [3]:
#Generate profile of members from roster
profile={}
members=[]
for tag in soup.find_all(tag_BIO):
    name=[string for string in tag.stripped_strings][0]
    url=root+tag['href']
    party=re.sub('\s+$','',re.sub('^\s*','',tag.parent.next_sibling))
    title=[string for string in tag.parent.find_previous_sibling(True).stripped_strings][0]
    members.append({'name':name,'title':title,'party':party,'url':url})

In [4]:
#From Bio page, personal information
def tag_BORN(tag):
    m=[re.search(r"OCCUPATION:",string) for string in tag.stripped_strings]
    return m and not None in m

for member in members:
    url=member['url']
    html=re.sub('<br/>|<br>\n*','',requests.get(url).text)
    soup=BeautifulSoup(html,"html5lib")
    profile_def={"birthday":"BORN:","occupation":"OCCUPATION:","political experience":"PUBLIC/PARTY SERVICE"}
    
    for key in profile_def:
        def tag_verify(tag):
            m=[re.search(profile_def[key],string) for string in tag.stripped_strings]
            return m and not None in m
        tag=soup.find(tag_verify,string=True)
        if tag==None:
            member[key]='N/A'
        else:
            tag_bday=tag.find_next_sibling('td')
            member[key]=[string for string in tag_bday.stripped_strings][0]

In [5]:
#Create Dataform in Panda using members list.
df=''
import pandas as pd
df=pd.DataFrame(members)
column_titles=['name','title','party','birthday','occupation','political experience','url']
df=df[column_titles]
df.to_excel('pd_to_excel.xlsx',index=False)

In [25]:
#Adjust formatting of Excel file
import openpyxl
from openpyxl.styles import Alignment
wb=openpyxl.load_workbook('pd_to_excel.xlsx')
sheet=wb.active
sheet.title='NJ Legislature'

#Set column widths
column_width={'A':25,'B':17,'C':11,'D':18,'E':30,'F':30,'G':53}
for column in column_width:
    sheet.column_dimensions[column].width=column_width[column]

#Top justify and wrap text for all cells
for column in [column_obj for column_obj in sheet.columns]:
    for cell in column:
        cell.alignment=Alignment(vertical='top', wrap_text=True)
    
#Freeze column headers
sheet.freeze_panes='A2'

#Save workbook
wb.save('pd_to_excel.xlsx')